In [3]:
%load_ext Cython

In [4]:
import numpy as np

In [9]:
%%cython
# distutils: language = c++
# distutils: source = power_method.cpp
# cython: language_level = 3
# cython: boundscheck = False
# cython: wraparound = False

from libcpp.vector cimport vector

cdef extern from "power_method.cpp":
    struct usv:
        const vector[vector[double]] U
        const vector[double] S
        const vector[vector[double]] V
